<a href="https://colab.research.google.com/github/SrMouraSilva/Introducao-a-Modelagem/blob/main/3_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Programação simbólica
from sympy import *
from sympy.plotting import plot, plot3d
 
# Exibir símbolos nos plots
from google.colab.output._publish import javascript
url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/3.1.2/latest.js?config=default"
 
javascript(url=url)
from IPython.display import Math
 
# Manipulação dos dados
import numpy as np
import pandas as pd

# Gráficos
import matplotlib as mpl
import matplotlib.pyplot as plt
 
import seaborn as sns
sns.set_theme()
 
mpl.rcParams['figure.figsize'] = (12,8)
mpl.rcParams['font.size'] = 14
mpl.rcParams['legend.fontsize'] = 14

## Questão 3.7
 
Reconsider the color TV problem of the Example 2.1, but now **use numerical methods** instead of the analytic methos we employed in Chapter 2.
 


### a) Determine the production levels $x_1$ and $x_2$ that maximize the objective function $y=f(x_1, x_2)$ in Eq. (2.2) of Chapter 2. Use the two–variable version of Newton’s method.
 
Relembrando o exemplo 2.1
 
> **Example 2.1.** A manufacturer of color TV sets is planning the introduction of two new products, a 19–inch LCD flat panel set with a manufacturer's suggested retail price (MSRP) of \\$339 and a 21–inch LCD flat panel set with an MSRP of \\$399. The cost to the company is $195 per 19–inch set and \\$225 per 21–inch set, plus an additional \\$400,000 in fixed costs. In the competitive market in which these sets will be sold, the number of sales per year will affect the average selling price. It is estimated that for each type of set, the average selling price drops by one cent for each additional unit sold. Furthermore, sales of the 19–inch set will affect sales of the 21–inch set, and vice–versa. It is estimated that the average selling price for the 19–inch set will be reduced by an additional 0.3 cents for each 21–inch set sold, and the price for the 21–inch set will decrease by 0.4 cents for each 19–inch set sold. How many units of each type of set should be manufactured?


 
#### Passo 1 - Variáveis
 
* $s$ = número de televisões de 19 polegadas vendidas (por ano)
* $t$ = número de televisões de 21 polegadas vendidas (por ano)
* $p$ = preço de venda de uma televisão de 19 polegadas (\\$)
* $q$ = preço de venda de uma televisão de 21 polegadas (\\$)
* $C$ = custo de manufatura das televisões (\\$)
* $R$ = receita obtida com a venda das televisões (\\$)
* $P$ = lucro com a venda das televisões (\\$)
 


#### Passo 1 - Suposições do problema

In [ ]:
s, t = var('s, t', real=True)

p = 339 - 0.01*s - 0.003*t
q = 399 - 0.004*s - 0.01*t
R = p*s + q*t
C = 400_000 + 195*s +225*t
P = R-C

#s >= 0
#t >= 0

#### Passo 2 - Abordagem de modelagem

Problema de otimização multidimensional cujo objetivo é maximizar $s$ e $t$. A solução se dará pelo método numérico de Newton.


#### Passo 3 - Formulação do modelo

$$
\begin{align}
\text{max} ~~& f(s, t) \\
\text{sujeito a} ~~& t \in \Omega \\
\end{align}
$$

Sendo:
* $f(s, t)$: Função objetivo
* $\Omega$: Espaço viável das variáveis do problema.

Desta forma, temos:

$$
\begin{align}
\text{max} ~~& f(s, t) = R-C \\
\text{sujeito a} ~~ & s \geq 0 \\
                    & t \geq 0
\end{align}
$$



#### Passo 4 - Resolução do modelo

Para aplicar o método de Newton, necessitamos de sua derivada primeira, segunda e da inversa da segunda. Seja a função $f(s, t)$ sendo

In [ ]:
f = P

Math(f"f(s, t) = {latex(f)}")

<IPython.core.display.Math object>

Temos o seu vetor gradiente 
$$\nabla f(s, t) = 
\left[
\begin{matrix}
    \frac{\partial f(s, t)}{\partial s} \\
    \frac{\partial f(s, t)}{\partial t}
\end{matrix}
\right]
$$
como

In [ ]:
grad_f = Matrix(derive_by_array(f, (s, t)))

display(Math(f"\\nabla f(s, t) = {latex(grad_f)}"))

<IPython.core.display.Math object>

Sua hessiana $$H = \nabla^2 f(s, t) = 
\left[
\begin{matrix}
    \frac{\partial^2 f(s, t)}{\partial s^2} & \frac{\partial^2 f(s, t)}{\partial s \partial t} \\
    \frac{\partial^2 f(s, t)}{\partial t \partial s} & \frac{\partial^2 f(s, t)}{\partial t^2}
\end{matrix}
\right]
$$
como

In [ ]:
hessian_f = hessian(f, [s, t])

Math(f"H = \\nabla^2 f(s, t) = {latex(hessian_f)}")

<IPython.core.display.Math object>

E a matriz inversa da hessiana como

In [ ]:
Math(f"H^{{-1}} = \\nabla^2 f(s, t) = {latex(hessian_f.inv())}")

<IPython.core.display.Math object>

Agora podemos executar o método de Newton. Após o código, encontra-se uma tabela exibindo os valores de $s$, $t$ e de $f(s, t)$ para cada $i$-ésima iteração, sendo $i$ informado na primeira coluna.

In [ ]:
def MetodoNewton(variaveis, valores_iniciais, funcao, numero_iteracoes=10):
    num_variaveis = len(variaveis)
    x_ = np.zeros((num_variaveis, numero_iteracoes+1))
    
    x_[:, 0] = valores_iniciais

    f = lambdify(variaveis, funcao, 'numpy')

    gradiente = Matrix(derive_by_array(funcao, variaveis))
    gradiente_f_original = lambdify(variaveis, gradiente, 'numpy')
    gradiente_f = lambda vetor: gradiente_f_original(*vetor.T.flatten())

    hessiana = hessian(funcao, variaveis)

    hessiana_inversa_f_original = lambdify(variaveis, hessiana.inv(), 'numpy')
    hessiana_inversa_f = lambda vetor: hessiana_inversa_f_original(*vetor.T.flatten())
    
    for k in range(numero_iteracoes):
        x_k = x_[:, [k]]

        x_k_next = x_k - hessiana_inversa_f(x_k) @ gradiente_f(x_k)
        x_[:, k+1] = x_k_next.T
    
    retorno = {f'{variavel}': x_[i] for i, variavel in enumerate(variaveis)}
    retorno[f'f{Tuple(*variaveis)}'] = [f(*x_i) for x_i in x_.T]
    
    return pd.DataFrame(retorno)

resultado_newton_a = MetodoNewton(
    variaveis=(s, t),
    valores_iniciais=(0, 0),
    funcao=f,
    numero_iteracoes=4
)

resultado_newton_a

,s,t,"f(s, t)"
0,0.000000,0.000000,-400000.000000
1,4735.042735,7042.735043,553641.025641
2,4735.042735,7042.735043,553641.025641
3,4735.042735,7042.735043,553641.025641
4,4735.042735,7042.735043,553641.025641


#### Passo 5 - Resposta da pergunta

A resposta obtida pelo modelo matemático proposto é que deve-se vender aproximadamente 4735 televisões de 19 polegadas e 7042 televisões de 21 polegadas, obtendo o lucro de \$553,641.

### b) As in Section 2.1, let $a$ denote the price elasticity for 19–inch sets. In part (a) we assumed $a= 0.01$. Now assume that $a$ increases by $10\%$ to $a = 0.011$ and repeat the optimization problem in part (a). Use your results to obtain a numerical estimate of the sensitivities $S(x_1, a), S(x_2, a)$, and $S(y, a)$. Compare to the results obtained analytically in Section 2.1.

Redefiniremos as variáveis, incluindo $a$ agora.


In [ ]:
s, t, a = var('s, t, a', real=True)

p = 339 - a*s - 0.003*t
q = 399 - 0.004*s - 0.01*t
R = p*s + q*t
C = 400_000 + 195*s +225*t
P = R-C

#s >= 0
#t >= 0

A função $f(s, t; a)$ será

In [ ]:
f = P

Math(f"f(s, t; a) = {latex(f)}")

<IPython.core.display.Math object>

Para este item, $a=0.011$, então executaremos novamente o método de Newton, definindo explicitamente que $a=0.011$. Deste modo, teremos $f(s, t)$ como sendo:

In [ ]:
#@title Valor de `a` { vertical-output: true, display-mode: "form" }
 
valor_a = 0.011 #@param {type:"number"}
 
variaveis = {
    a: valor_a
}
 
Math(f"f(s, t) = f(s, t; a={valor_a}) = {latex(f.subs(variaveis))}")

<IPython.core.display.Math object>

O resultado do método de Newton é apresentado na tabela abaixo. Os valores $s$, $t$ e $f(s, t)$ são apresentados para cada $i$-ésima iteração ($i$-ésima linha).

In [ ]:
resultado_newton_b = MetodoNewton(
    variaveis=(s, t),
    valores_iniciais=(0, 0),
    funcao=f.subs(variaveis),
    numero_iteracoes=4
)
resultado_newton_b

,s,t,"f(s, t)"
0,0.000000,0.000000,-400000.000000
1,4250.639386,7212.276215,533514.066496
2,4250.639386,7212.276215,533514.066496
3,4250.639386,7212.276215,533514.066496
4,4250.639386,7212.276215,533514.066496


Podemos obter uma estimativa das sensibilidades comparando o resulta do item a com este item. Para calcular $S(x_i, a)$, a sensibilidade de $x_i$ em função de $a$, façamos
$$
S(x_i, a) = \frac{x_i^{*_b}}{x_i^{*_a}} - 1,
$$
sendo $x_i^{*_n}$ a $i$-ésima variável ótima no item $n$ desta questão e
$$
S(y, a) = \frac{f(s, t; a=0.011)}{f(s, t; a=0.01)} - 1.
$$

In [ ]:
valor_s_a, valor_t_a, valor_f_a = resultado_newton_a.iloc[-1]
valor_s_b, valor_t_b, valor_f_b = resultado_newton_b.iloc[-1]


Math(f"""
\\begin{{aligned}}
S(s, a) &= \\frac{{{valor_s_b}}}{{{valor_s_a}}} - 1 = {(valor_s_b/valor_s_a - 1) * 100}\%\\\\
S(t, a) &= \\frac{{{valor_t_b}}}{{{valor_t_a}}} - 1 = {(valor_t_b/valor_t_a - 1) * 100}\%\\\\
S(y, a)   &= \\frac{{{valor_f_b}}}{{{valor_f_a}}} - 1 = {(valor_f_b/valor_f_a - 1) * 100}\%
\\end{{aligned}}
""")

<IPython.core.display.Math object>

Desta forma, temos que, a medida que $a$ aumenta $10\%$, para o ponto ótimo
* $x_1$, o número de vendas das televisões de 19 polegadas, cai $10.230\%$
* $x_2$, o número de vendas das televisões de 21 polegadas, aumenta $2.407\%$
* $y$, o lucro cai $3.635\%$

Já no exemplo 2.1, a medida que $a$ aumenta $10\%$, para o ponto ótimo
* $x_1$, o número de vendas das televisões de 19 polegadas, cai $11.0\%$
* $x_2$, o número de vendas das televisões de 21 polegadas, aumenta $2.7\%$
* $y$, o lucro, cai $4\%$. 

Observe que a análise de sensibilidade numérica das variáveis $x_1$, $x_2$ e $y$ é promissora se comparada com a análise de sensibilidade analítica das variáveis $x_1$, $x_2$ e $y$ a medida que $a$ aumenta $10\%$, porém vale ressaltar que o erro relativo de cada análise de sensibilidade numérica  para cada variável ainda está distante de zero. Veja logo abaixo
* $r_{x_1} = \frac{0.11 - 0.1023}{0.11} = 0.07$ 
* $r_{x_2} = \frac{0.027 - 0.02407}{0.027} = 0.1085$
* $r_{y} = \frac{0.04 - 0.0365}{0.04} = 0.0875$




### c) Let $b$ denote the price elasticity for 21–inch sets. Currently, $b= 0.01$. As in part (b), use numerical methods to estimate the sensitivities of $x_1,x_2$, and $y$ to the parameter $b$.

Redefinamos as variáveis, incluindo $b$ agora.



In [ ]:
s, t, b = var('s, t, b', real=True)

p = 339 - 0.01*s - 0.003*t
q = 399 - 0.004*s - b*t
R = p*s + q*t
C = 400_000 + 195*s +225*t
P = R-C

#s >= 0
#t >= 0

A função $f(s, t; b)$ será

In [ ]:
f = P

Math(f"f(s, t; b) = {latex(f)}")

<IPython.core.display.Math object>

Para este item, $b=0.011$, então executaremos novamente o método de Newton, definindo explicitamente que $b=0.011$. Deste modo, teremos $f(s, t)$ como sendo:

In [ ]:
#@title Valor de `b` { vertical-output: true, display-mode: "form" }

valor_b = 0.011 #@param {type:"number"}

variaveis = {
    b: valor_b
}

Math(f"f(s, t) = f(s, t; b={valor_b}) = {latex(f.subs(variaveis))}")

<IPython.core.display.Math object>

O resultado do método de Newton é apresentado na tabela abaixo. Os valores $s$, $t$ e $f(s, t)$ são apresentados para cada i-ésima iteração ($i$-ésima linha).

In [ ]:
resultado_newton_c = MetodoNewton(
    variaveis=(s, t),
    valores_iniciais=(0, 0),
    funcao=f.subs(variaveis),
    numero_iteracoes=4
)
resultado_newton_c

,s,t,"f(s, t)"
0,0.000000,0.000000,-400000.000000
1,4987.212276,6322.250639,509115.089514
2,4987.212276,6322.250639,509115.089514
3,4987.212276,6322.250639,509115.089514
4,4987.212276,6322.250639,509115.089514


Podemos obter uma estimativa das sensibilidades, comparando o resultado do item a com este item. Para calcular $S(x_1, b)$, a sensibilidade de $x_1$ em função de $b$, façamos
$$
S(x_1, b) = \frac{x_1^{*_b}}{x_1^{*_a}} - 1,
$$
sendo $x_i^{*_n}$ a $i$-ésima variável ótima no item $n$ desta questão e
$$
S(y, b) = \frac{f(s, t; b=0.011)}{f(s, t; b=0.01)} - 1.
$$

In [ ]:
valor_s_a, valor_t_a, valor_f_a = resultado_newton_a.iloc[-1]
valor_s_c, valor_t_c, valor_f_c = resultado_newton_c.iloc[-1]


Math(f"""
\\begin{{aligned}}
S(x_1, b) &= \\frac{{{valor_s_c}}}{{{valor_s_a}}} - 1 = {(valor_s_c/valor_s_a - 1) * 100}\%\\\\
S(x_2, b) &= \\frac{{{valor_t_c}}}{{{valor_t_a}}} - 1 = {(valor_t_c/valor_t_a - 1) * 100}\%\\\\
S(y, b)   &= \\frac{{{valor_f_c}}}{{{valor_f_a}}} - 1 = {(valor_f_c/valor_f_a - 1) * 100}\%
\\end{{aligned}}
""")

<IPython.core.display.Math object>

Desta forma, temos que a medida que $b$ aumenta $10\%$, para o ponto ótimo
* $x_1$, o número de vendas das televisões de 19 polegadas, aumenta $5.326\%$
* $x_2$, o número de vendas das televisões de 21 polegadas, cai $10.230\%$
* $y$, o lucro, cai $8.042\%$

### d) Compare the analytic methods of Section 2.1 to the numerical methods employed in this problem. Which do you prefer? Explain.

Conforme foi visto no item b), a análise de sensibilidade numérica foi promisora se comparada com a análise de sensibilidade analítica, porém o erro relativo ainda estava distante de zero. Logo, preferimos utilizar métodos analíticos do que métodos numéricos caso seja possível utilizar recursos computacionais para a obtenção de resultados analíticos para não haver problemas de erro numérico como foi visto no item b) desta questão.